In [ ]:
import pandas as pd
import numpy as np
import qgridnext as qgrid 
from IPython.display import display, clear_output
import ipywidgets as widgets

# Step 1: Create a DataFrame with Multi-Index Columns
data = np.random.randn(4, 8)

columns = pd.MultiIndex.from_product(
    [['Category1', 'Category2'], ['SubCat1', 'SubCat2'], ['Detail1', 'Detail2']],
    names=['Category', 'SubCategory', 'Detail']
)

df = pd.DataFrame(data, index=['Row1', 'Row2', 'Row3', 'Row4'], columns=columns)
# Display the original DataFrame
print("Original DataFrame with Multi-Index Columns:")
display(df)
print("Original DataFrame Index:", df.index.tolist())
print("Original DataFrame Columns:", df.columns.tolist())

# Step 2: Convert Multi-Index Columns to Rows for qgrid
# Extracting the level values and creating a DataFrame for each level
levels = df.columns.names
level_values = [df.columns.get_level_values(i) for i in range(len(levels))]

# Create a DataFrame for the levels
level_dfs = []
for i, level_value in enumerate(level_values):
    level_df = pd.DataFrame([level_value], columns=df.columns)
    level_df.index = [levels[i]]
    level_dfs.append(level_df)

# Create a DataFrame for the values
value_df = pd.DataFrame(data, columns=df.columns, index=df.index)

# Concatenate the levels and values DataFrames
combined_df = pd.concat(level_dfs + [value_df])

# Transpose the DataFrame to fit qgrid format
combined_df = combined_df.T

# Print the transposed DataFrame's index and columns for clarity
print("Transposed DataFrame Index:", combined_df.index.tolist())
print("Transposed DataFrame Columns:", combined_df.columns.tolist())

# Step 3: Ensure unique column names with modifications to avoid duplications
# Adjusting column names to prepend "_" for MultiIndex names
new_columns = []
for i, col in enumerate(combined_df.columns):
    if col in levels:  # Check if the column name is in the original MultiIndex names
        new_columns.append(f'_{col}')  # Prepend "_" to the original name
    else:
        new_columns.append(f'{col}')  # Use the modified name with suffix for unique columns

combined_df.columns = new_columns

# Adjusting row names to append "_" for MultiIndex names
# This is done during the creation of level_dfs
level_dfs_adjusted = []
for i, level_df in enumerate(level_dfs):
    adjusted_index = [f'{index}_' for index in level_df.index]  # Append "_" to each index name
    level_df.index = adjusted_index
    level_dfs_adjusted.append(level_df)

# Recreate the combined DataFrame with adjusted level DataFrames
combined_df_adjusted = pd.concat(level_dfs_adjusted + [value_df]).T

# Now, combined_df_adjusted has "_" prepended to column names that are part of the original MultiIndex
# and "_" appended to row names that are part of the original MultiIndex

# Display the DataFrame with separated levels
print("Combined DataFrame for qgrid:")
display(combined_df)
print("Combined DataFrame Index after adjustments:", combined_df.index.tolist())
print("Combined DataFrame Columns after adjustments:", combined_df.columns.tolist())


# Step 1: Identify columns to keep (those without an underscore)
columns_to_keep = [col for col in combined_df.columns if '_' not in col]

# Step 2: Filter out unnecessary columns
df_cleaned = combined_df[columns_to_keep]

# Step 4: Create qgrid widget and output area for displaying filtered DataFrame
qgrid_widget = qgrid.show_grid(df_cleaned, show_toolbar=True)
output_area = widgets.Output()

# Event handler to update the displayed DataFrame whenever changes are made in qgrid
def on_filter_change(event, widget):
    with output_area:
        clear_output(wait=True)
        filtered_df = widget.get_changed_df().T
        display(filtered_df)

# Attach the event handler to the qgrid widget
qgrid_widget.on('filter_changed', on_filter_change)

# Step 5: Display the qgrid widget and the output area
display(qgrid_widget)
display(output_area)

In [ ]:
import pandas as pd
import numpy as np
import qgridnext as qgrid
import json
from IPython.display import display, clear_output
import ipywidgets as widgets

class MultiIndexDataFrameToQGrid:
    def __init__(self, df):
        self.df = df
        self.combined_df = None
        self.df_cleaned = None
        self.qgrid_widget = None
        self.output_area = widgets.Output()

    def transpose_and_prepare_df(self):
        # Extracting the level values and creating a DataFrame for each level
        levels = self.df.columns.names
        level_values = [self.df.columns.get_level_values(i) for i in range(len(levels))]

        # Create a DataFrame for the levels
        level_dfs = []
        for i, level_value in enumerate(level_values):
            level_df = pd.DataFrame([level_value], columns=self.df.columns)
            level_df.index = [levels[i]]
            level_dfs.append(level_df)

        # Create a DataFrame for the values
        value_df = pd.DataFrame(self.df.values, columns=self.df.columns, index=self.df.index)

        # Concatenate the levels and values DataFrames and transpose
        self.combined_df = pd.concat(level_dfs + [value_df]).T

        # Adjusting column and row names
        new_columns = [col if col not in levels else f'_{col}' for col in self.combined_df.columns]
        self.combined_df.columns = new_columns

        # Filter out unnecessary columns
        columns_to_keep = [col for col in self.combined_df.columns if '_' not in col]
        self.df_cleaned = self.combined_df[columns_to_keep]

    def display_in_qgrid(self):
        # Create qgrid widget for displaying filtered DataFrame
        self.qgrid_widget = qgrid.show_grid(self.df_cleaned, show_toolbar=True)

        # Attach event handler to the qgrid widget
        self.qgrid_widget.on('filter_changed', self.on_filter_change)

        # Display the qgrid widget and the output area
        display(self.qgrid_widget)
        display(self.output_area)

    def on_filter_change(self, event, widget):
        with self.output_area:
            clear_output(wait=True)
            filtered_df = widget.get_changed_df().T
            display(filtered_df)

    
    def write_dataframe_to_json_with_multiindex(self, json_file_path):
        df = self.df
        # Prepare data to include MultiIndex column structure
        data = {
            'data': df.to_json(orient='split', indent=4),
            'columns': df.columns.tolist()
        }
        
        # Serialize the structure to JSON file
        with open(json_file_path, 'w') as file:
            json.dump(data, file, indent=4)
        
    def read_json_as_dataframe_with_multiindex(self, json_file_path):
        # Load the structure from JSON file
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        
        # Reconstruct DataFrame from 'data' part
        df_data = pd.read_json(data['data'], orient='split')
        
        # Reconstruct MultiIndex for columns
        multiindex_columns = pd.MultiIndex.from_tuples(data['columns'])
        df_data.columns = multiindex_columns
        
        return df_data

    

In [ ]:
# Example usage
data = np.random.randn(4, 8).astype(int)
columns = pd.MultiIndex.from_product([['Creatures', 'Spells'], ['Mage', 'Warrior'], ['Input', 'Output']], names=['Category', 'SubCategory', 'Detail'])
df = pd.DataFrame(data, index=['DeckA', 'DeckB', 'DeckC', 'DeckD'], columns=columns)

df.to_csv("dataframe.csv", header=True, index=True, sep=";")
# Initialize the class with the DataFrame
converter = MultiIndexDataFrameToQGrid(df)

converter.write_dataframe_to_json_with_multiindex("dataframe.json")
#converter.read_json_as_dataframe_with_multiindex("dataframe.json")

# Prepare and transpose the DataFrame
converter.transpose_and_prepare_df()

# Display the DataFrame in qgrid
converter.display_in_qgrid()



In [ ]:
import gspread  
from oauth2client.service_account import ServiceAccountCredentials  
  
# Define the scope and credentials  
scope = ['https://www.googleapis.com/auth/spreadsheets']  
credentials = ServiceAccountCredentials.from_json_keyfile_name('precise-armor-426813-v6-3c3c5c85f7df.json', scope)  
  
# Authorize the client using the credentials  
client = gspread.authorize(credentials)  
  
# Open the Google Sheet by its key  
sheet = client.open_by_key('1RSo-kmJVzam-lVXOsA8N_fKBDDhjyqXFRGjqXkZ_rUQ')  
  
# Select the first worksheet  
worksheet = sheet.worksheet("Card Database")  
  
# Get all values from the worksheet  
values = worksheet.get_all_values()  

print(values)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the widgets for the Data Input pane
data_input_form = widgets.VBox([
    widgets.Text(description="Name:", placeholder="Enter your name"),
    widgets.IntSlider(description="Age:", value=25, min=0, max=100, step=1),
    widgets.Textarea(description="Bio:", placeholder="Enter a short biography")
])

# Define the widgets for the Visualization Controls pane
visualization_controls = widgets.VBox([
    widgets.Dropdown(options=['Pie Chart', 'Bar Graph', 'Line Plot'], description='Chart Type:'),
    widgets.Checkbox(value=True, description='Show Legend')
])

# Create the Tab widget with two children
tab = widgets.Tab(children=[data_input_form, visualization_controls])
tab.set_title(0, 'Data Input')
tab.set_title(1, 'Visualization Controls')

# Display the Tab widget
display(tab)

# Define a function to handle changes in the Visualization Controls
def update_visualization(change):
    chart_type = visualization_controls.children[0].value
    show_legend = visualization_controls.children[1].value
    print(f"Update the plot to a {chart_type} with legend set to {show_legend}")

# Attach the event handler to the Dropdown and Checkbox widgets
visualization_controls.children[0].observe(update_visualization, names='value')
visualization_controls.children[1].observe(update_visualization, names='value')

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create output widgets for each tab
tab_output1 = widgets.Output()
tab_output2 = widgets.Output()

# Create tab widget with these outputs as children
tab = widgets.Tab(children=[tab_output1, tab_output2])
tab.set_title(0, 'Grids in Tab 1')
tab.set_title(1, 'Grids in Tab 2')
display(tab)

# Assume grids or other content need to be displayed in these tabs
def display_grid_in_tab(grid, tab_output):
    """Displays a grid within a specified tab's output widget."""
    with tab_output:
        clear_output(wait=True)  # Clear existing content
        display(grid)  # Display the new grid

# Example grid widgets (can be any widget or content)
grid1 = widgets.Label('Grid 1: Content for Tab 1')
grid2 = widgets.Label('Grid 2: Content for Tab 2')

# Initially display grids in their respective tabs
display_grid_in_tab(grid1, tab_output1)
display_grid_in_tab(grid2, tab_output2)

# Function to redraw grid in a specific tab (can be triggered by a button or other events)
def redraw_grid_in_tab():
    new_grid1 = widgets.Label('Grid 1: Refreshed Content for Tab 1')
    new_grid2 = widgets.Label('Grid 2: Refreshed Content for Tab 2')
    display_grid_in_tab(new_grid1, tab_output1)
    display_grid_in_tab(new_grid2, tab_output2)

# Optionally, add a button to trigger redrawing grids
redraw_button = widgets.Button(description="Redraw Grids")
redraw_button.on_click(lambda b: redraw_grid_in_tab())
display(redraw_button)

In [ ]:
import ipywidgets as widgets

# Define widgets
large_dropdown = widgets.SelectMultiple(
    options=['Option 1', 'Option 2', 'Option 3', 'Option 4', 'Option 5', 'Option 6'],
    rows=10,  # Larger widget
    description=''  # Remove description to control label placement manually
)

small_dropdown = widgets.SelectMultiple(
    options=['Option 1', 'Option 2', 'Option 3'],
    rows=3,  # Smaller widget
    description=''  # Remove description to control label placement manually
)

# Create GridspecLayout with 2 rows and multiple columns
num_widgets = 2  # You can increase this number based on how many widgets you have
grid = widgets.GridspecLayout(2, num_widgets, height='auto', width='100%')

# Set labels in the first row
grid[0, 0] = widgets.Label('Large Multi-select:')
grid[0, 1] = widgets.Label('Small Multi-select:')

# Set widgets in the second row
grid[1, 0] = large_dropdown
grid[1, 1] = small_dropdown

display(grid)
# Create the same widgets and labels
labels = [
    widgets.Label('Large Multi-select:'),
    widgets.Label('Small Multi-select:')
]
widgets_list = [large_dropdown, small_dropdown]

# Create GridBox with labels on the first row and widgets on the second row
grid_box = widgets.GridBox(
    children=labels + widgets_list,  # Concatenate labels and widgets in the list
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='repeat(2, 1fr)',  # Adjust number of columns as needed
        grid_template_rows='auto auto',  # Two rows: one for labels, one for widgets
        grid_gap='10px'
    )
)

display(grid_box)

In [ ]:
import pandas as pd
import json
import os
import qgrid
from IPython.display import display, clear_output
import ipywidgets as widgets
from io import StringIO

class MultiIndexDataFrameToQGrid():
    def __init__(self, df=None):
        self.df = df
        self.combined_df = None
        self.df_cleaned = None
        self.qgrid_widget = None
        self.output_area = widgets.Output()

    def transpose_and_prepare_df(self):
        # Extracting the level values and creating a DataFrame for each level
        levels = self.df.columns.names
        level_values = [self.df.columns.get_level_values(i) for i in range(len(levels))]

        # Create a DataFrame for the levels
        level_dfs = []
        for i, level_value in enumerate(level_values):
            level_df = pd.DataFrame([level_value], columns=self.df.columns)
            level_df.index = [levels[i]]
            level_dfs.append(level_df)

        # Create a DataFrame for the values
        value_df = pd.DataFrame(self.df.values, columns=self.df.columns, index=self.df.index)

        # Concatenate the levels and values DataFrames and transpose
        self.combined_df = pd.concat(level_dfs + [value_df]).T

        # Adjusting column and row names
        new_columns = [col if col not in levels else f'_{col}' for col in self.combined_df.columns]
        self.combined_df.columns = new_columns

        # Filter out unnecessary columns
        columns_to_keep = [col for col in self.combined_df.columns if '_' not in col]
        self.df_cleaned = self.combined_df[columns_to_keep]

    def display_in_qgrid_new(self):
        if self.df_cleaned is not None:
            # Ensure column names are tuples, not lists
            self.df_cleaned.columns = pd.MultiIndex.from_tuples(
                tuple(col) if isinstance(col, list) else col for col in self.df_cleaned.columns
            )

            # Create qgrid widget for displaying filtered DataFrame
            self.qgrid_widget = qgrid.show_grid(self.df_cleaned, show_toolbar=True)

            # Attach event handler to the qgrid widget
            self.qgrid_widget.on('filter_changed', self.on_filter_change)

            # Display the qgrid widget and the output area
            display(self.qgrid_widget)
            display(self.output_area)

    def display_in_qgrid(self):
        # Create qgrid widget for displaying filtered DataFrame
        self.qgrid_widget = qgrid.show_grid(self.df_cleaned, show_toolbar=True)

        # Attach event handler to the qgrid widget
        self.qgrid_widget.on('filter_changed', self.on_filter_change)

        # Display the qgrid widget and the output area
        display(self.qgrid_widget)
        display(self.output_area)

    def on_filter_change(self, event, widget):
        with self.output_area:
            clear_output(wait=True)
            filtered_df = widget.get_changed_df().T
            display(filtered_df)
            
    def write_dataframe(self, df, file_path, delimiter=';'):
        if df is None:
            raise ValueError("Provided DataFrame is None, cannot serialize to file.")
        if file_path.endswith('.json'):
            # JSON: serialize DataFrame with MultiIndex for both columns and index
            data = {
                'data': df.to_json(orient='split'),  # Contains data along with regular index and columns
                'columns': [list(col) if isinstance(col, tuple) else col for col in df.columns.tolist()],  # Handle MultiIndex columns
                'index': [list(idx) if isinstance(idx, tuple) else idx for idx in df.index.tolist()] if isinstance(df.index, pd.MultiIndex) else df.index.tolist()  # Handle MultiIndex index
            }
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
        elif file_path.endswith('.csv'):
            # CSV: write DataFrame ensuring MultiIndex is preserved
            df.to_csv(file_path, header=True, index=True, sep=delimiter)

    def read_dataframe(self, file_path, delimiter=';'):
        if file_path.endswith('.json'):
            with open(file_path, 'r') as file:
                data = json.load(file)
            
            # Manually parse the data section of the JSON
            # The 'data' in JSON corresponds to DataFrame values and 'index' to DataFrame index
            df_data = pd.DataFrame(data['data'], index=data['index'])
            
            # Apply column names from JSON, reconstructing MultiIndex if necessary
            if isinstance(data['columns'][0], list):
                # Reconstruct MultiIndex from list of tuples
                multiindex_columns = pd.MultiIndex.from_tuples(data['columns'])
                df_data.columns = multiindex_columns
            else:
                # Apply columns as normal if they are not in a list of tuples
                df_data.columns = data['columns']
            
            self.df = df_data
            return df_data
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path, header=[0, 1], index_col=[0, 1], sep=delimiter)
            self.df = df
            return df

    def getWidgets(self):
        return self.qgrid_widget, self.output_area

multiIndexDataFrame = MultiIndexDataFrameToQGrid()

csv_file_path = 'multiindex_dataframe.csv'

# Read from CSV
df_from_csv = multiIndexDataFrame.read_dataframe(csv_file_path)
print("From CSV:", df_from_csv)

multiIndexDataFrame.transpose_and_prepare_df()
multiIndexDataFrame.display_in_qgrid()


In [ ]:
multiIndexDataFrame = MultiIndexDataFrameToQGrid()

csv_file_path = 'multiindex_dataframe.csv'

# Read from CSV
df_from_csv = multiIndexDataFrame.read_dataframe(csv_file_path)
print("From CSV:", df_from_csv)

multiIndexDataFrame.transpose_and_prepare_df()
multiIndexDataFrame.display_in_qgrid()

In [ ]:
from IPython.display import HTML
import qgridnext as qgrid
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({
    'Column1': ['A', 'B', 'C', 'D'],
    'Column2': [1, 2, 3, 4],
    'Column3': [5, 6, 7, 8],
    'Column4': [9, 10, 11, 12],
})

# Create the qgrid widget
qgrid_widget = qgrid.show_grid(df, column_options={'width': 10}, grid_options={'forceFitColumns': False})

# Inject custom CSS to rotate the header text for all columns starting from the second column
display(HTML("""
<style>
    /* Apply rotation to the header of all columns starting from the 2nd column */
    .slick-header-column:nth-child(n+3) {
        height: 80px !important; /* Increase header height */
        vertical-align: bottom; /* Align the text to the bottom */
    }
    .slick-header-column:nth-child(n+3) .slick-column-name {
        transform: rotate(90deg);
        transform-origin: left bottom;
        white-space: nowrap;
        margin-left: 0px; /* Reduce spacing */
        margin-top: 0px; /* Adjust this value to fine-tune text placement */
    }

    .slick-cell {
        height: 40px; /* Adjust row height to match rotated header */
    }
</style>
"""))

# Display the qgrid widget
display(qgrid_widget)

In [8]:
from IPython.display import HTML, display

def apply_rotation_css_to_qgrid(qgrid_widget, start_column=2, header_height=80, row_height=40, custom_class='custom-qgrid'):
    """
    Applies CSS to rotate the headers of specific columns in a qgrid widget.

    Parameters:
    - qgrid_widget: The qgrid widget to which the CSS will be applied.
    - start_column: The column number to start rotating headers from (1-based index).
    - header_height: The height of the header row (in pixels).
    - row_height: The height of the data rows (in pixels).
    - custom_class: The custom CSS class to be applied to the qgrid widget.
    """
    # Add a custom class to the qgrid widget
    qgrid_widget.add_class(custom_class)
    
    # Generate CSS for the rotation
    style = f"""
    <style>
        /* Apply rotation to the header of all columns starting from the specified column */
        .{custom_class} .slick-header-column:nth-child(n+{start_column}) {{
            height: {header_height}px !important; /* Set header height */
            vertical-align: bottom; /* Align the text to the bottom */
        }}
        .{custom_class} .slick-header-column:nth-child(n+{start_column}) .slick-column-name {{
            transform: rotate(90deg);
            transform-origin: left bottom;
            white-space: nowrap;
            margin-left: 0px; /* Adjust spacing */
            margin-top: 10px; /* Adjust text placement */
        }}
        .{custom_class} .slick-cell {{
            height: {row_height}px; /* Set row height to match rotated header */
        }}
    </style>
    """

    # Inject the CSS into the notebook
    display(HTML(style))

# Example usage
import qgrid
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({
    'Column1': ['A', 'B', 'C', 'D'],
    'Column2': [1, 2, 3, 4],
    'Column3': [5, 6, 7, 8],
})

# Create the qgrid widget
qgrid_widget = qgrid.show_grid(df, column_options={'width': 10}, grid_options={'forceFitColumns': False})

# Apply the rotation CSS starting from the second column
apply_rotation_css_to_qgrid(qgrid_widget, start_column=2)

# Display the qgrid widget
display(qgrid_widget)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…